# Architecture Breakdown of SAINT

The following notebook is designed to break down the different elements of the SAINT architecture as documentation for the development of the code, and the corresponding customizations. 



## Acknowledgments 

The content of the breakdown is based on the following public repos:
* **SAINT**: https://github.com/somepago/saint

Similarly, various utilities from the SAINT paper were borrowed from the following repo, that will also be acknowledged:
* **Tab Transformer PyTorch**: https://github.com/lucidrains/tab-transformer-pytorch



## SAINT Architecture

<img src="saint_architecture.png" alt="SAINT Architecture">


## Equations

The SAINT pipeline is described by the following equations found in section 3.1 of the paper. 

$$
\mathbf{z}_\mathbf{i}^{(1)} = \operatorname{LN}\Bigl(\operatorname{MSA}\bigl(\mathbf{E}(\mathbf{x}_\mathbf{i})\bigr)\Bigr) + \mathbf{E}(\mathbf{x}_\mathbf{i})
$$

$$
\mathbf{z}_\mathbf{i}^{(2)} = \operatorname{LN}\Bigl(\mathrm{FF}_1\bigl(\mathbf{z}_\mathbf{i}^{(1)}\bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(1)}
$$

$$
\mathbf{z}_\mathbf{i}^{(3)} = \operatorname{LN}\Bigl(\operatorname{MISA}\Bigl(\{\mathbf{z}_\mathbf{i}^{(2)}\}_{i=1}^b\Bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(2)}
$$

$$
\mathbf{r}_\mathbf{i} = \operatorname{LN}\Bigl(\mathrm{FF}_2\bigl(\mathbf{z}_\mathbf{i}^{(3)}\bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(3)}
$$

### SAINT Architecture Stages

* **Embedding function**: Converts the raw input $x_i$ into a higher-dimensional vector representation where for a given $\mathbf{x}_{\mathbf{i}} \in \mathbb{R}^{(n+1)}$, we get $\mathbf{E}\left(\mathbf{x}_{\mathbf{i}}\right) \in \mathbb{R}^{(n+1) \times d}$. It is important to covert all feature types into contextual embeddings and not separate them as in the Tab Transformers paper given that separating them prevents the continuous data from taking advantage of the self-attention block, meaning correlations between categorical and continuous data are lost. Developing a higher dimensional embedding where both feature types are included facilitates the data points to find better representations of each other.
* **Multi-head self-attention (MSA) module**: Module used to focus on multiple parts of the input simultaneously (`torch.nn.MultiHeadAttention()`)
* **Layer normalization (LN) layer**: Method of regularization to avoid overfitting (`torch.nn.LayerNorm()`)
* **Feed Forward (FF) layer**:  Fully connected network that introduces non-linearity and transforms token's representations through expansions and projections of the input's dimensions to learn meaningful relationships. 
* **Transformer Encoder**: The Transformer Encoder is a collection of the layers listed above. These are complemented by skip (or residual) connections inside the Transformer's encoder. This means that the inputs are fed directly to immediate layers as well as subsequent layers. This is beneficial because it allows that at the start of the training, when the gradient signal is still being initialized, the second signal goes through without any delay due to initialization. Hence, the lower layers can start training on this signal, and then incorporate the gradient once it is propagated through the block. Overall, it facilitates gradient flow, helps the model learn modifications rather than complete transformations, and stabilizes training.



### Equation 1 Overview

$$
\mathbf{z}_\mathbf{i}^{(1)} = \operatorname{LN}\Bigl(\operatorname{MSA}\bigl(\mathbf{E}(\mathbf{x}_\mathbf{i})\bigr)\Bigr) + \mathbf{E}(\mathbf{x}_\mathbf{i})
$$

This equation deals with the intra-sample self-attention to capture the relationships amongst the row's features. It analyzes row $x_i$ at a time from the tabular data. In both occasions, we covert the raw input $x_i$ into an embedding through the embedding $\mathbf{E}$ function which prepares the data for the attention mechanism. 

After embedding the row, we wrap it in a MSA module followed by a layer normalization (LN) layer. Finally, we add a skip/residual connection by adding the original embedding back to the LN layer. The connection helps deal with vanishing gradients, and ensures that if the attention mechanism does not provide useful values, the original signal is still retinaed. 

The output of this equation is the row's representation after the self-attention sub-layer. Each token represents its original embedding and the contextual influence of other features in the same row.  


### Equation 2 Overview

$$
\mathbf{z}_\mathbf{i}^{(2)} = \operatorname{LN}\Bigl(\mathrm{FF}_1\bigl(\mathbf{z}_\mathbf{i}^{(1)}\bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(1)}
$$

This equation continues equation 1's output which is the richer representation of the row's features. Each token is individually fed to the Feed Forward layer, which is then regularized by the LN layer. Similarly, the original $z_i$ is added to the normalized output as a skip/residual connection. 

The output of this equation is an even more refined and processed representation of the input. 


### Equation 3 Overview 

$$
\mathbf{z}_\mathbf{i}^{(3)} = \operatorname{LN}\Bigl(\operatorname{MISA}\Bigl(\{\mathbf{z}_\mathbf{i}^{(2)}\}_{i=1}^b\Bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(2)}
$$

While equations 1 and 2 worked individually on each feature, equation 3 uses each row $z_i$ that has been processed through the self-attention and the FF layer, and applies the attention mechanism across all rows in the batch of size `b`, hence, no longer within a single row. 

Similar to the MSA module, the MISA applies the self-attention method, but its domain it the set of all row representations in the batch. For each row $i$, MISA computes attention scores with respect to all other rows, leading to intersample relationships. 

After performing attention, MISA is wrapped in a LN layer for regularization. Once again, we finalize the section by adding the skip/residual connection from the original representation $z_i$.

The output of this equation is an enhanced representation that came through the intrasample relationships from equations 1 and 2, with now the intersample relationships. 

### Equation 4 Overview

$$
\mathbf{r}_\mathbf{i} = \operatorname{LN}\Bigl(\mathrm{FF}_2\bigl(\mathbf{z}_\mathbf{i}^{(3)}\bigr)\Bigr) + \mathbf{z}_\mathbf{i}^{(3)}
$$

The final equation of the SAINT architecture, it takes the output from equation 3 that benefited from self-attention, feed forward and intersample attention. 

It takes said input, and passes it through a FF layer, and then regularizes it through the LN layer. Finally, it adds once more the original representation $z_i$. 

The output of this equation incorporates:
* The original aggregated features
* Intrasample relationships through self-attention and FF layers
* Intersample relationships through row-level self-attention and FF layers
* Additional transformations

The output $r_i$ can be fed into the classification head or further processed within the architecture of this project. 

## Bringing the Equations together

Section 3.1 describes the SAINT architecture as follows

>SAINT is composed of a stack of L identical stages. Each stage consists of one self-attention
transformer block and one intersample attention transformer block. The self-attention transformer
block is identical to the encoder from [41]. It has a multi-head self-attention layer (MSA) (with h
heads), followed by two fully-connected feed-forward (FF) layers with a GELU non-linearity [16].
Each layer has a skip connection [14] and layer normalization (LN) [2]. The intersample attention
transformer block is similar to the self-attention transformer block, except that the self-attention layer
is replaced by an intersample attention layer (MISA). The details of the intersample attention layer
are presented in the following subsection.

## Paper Hyperparameters

The following hyper parameters yielded the best results for the SAINT architecture:
* $\beta_1=0.9, \beta_2=0.999$ for the `torch.optim.AdamW` optimizer (section 5.0)
* `weight_decay=0.01` (section 5.0)
* `batch_size = 256` although due to computation power this might be reduced (section 5.0)
* `lr=0.0001` (section 5.0)
* `dropout=0.1` in all attention layers and feed-forward layers (appendix C)